In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('Clean_Data.csv', sep=';', dtype= {'feat_class': 'object', 'feat_group_weeks': 'object'})
df.drop(['student_email', 'group_title', 'prep_email', 'date_first_l', 'student_id', 'group_id'], axis=1, inplace=True)

In [3]:
categorical_cols = df[['feat_class', 'feat_group_weeks', 'feat_payment_type', 'feat_payment_place', 'feat_group_format']].columns.tolist()
print("Категориальные столбцы для кодирования:", categorical_cols)
ohe = OneHotEncoder(
    sparse_output=False,      
    handle_unknown='ignore',
    drop='first'             # Избегаем мультиколлинеарности
)
# Создаем преобразователь столбцов
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical_cols)
    ],
    remainder='passthrough'  # че делаем с остальными столбцами
)
preprocessor.fit(df)
encoded_data = preprocessor.transform(df)
encoded_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
other_columns = [col for col in df.columns if col not in categorical_cols]
all_columns = np.concatenate([encoded_columns, other_columns])
encoded_df = pd.DataFrame(encoded_data, columns=all_columns)
encoded_df_withHWcat = encoded_df.copy()
encoded_df_withHWcat.drop('feat_good_agr_before', axis=1, inplace=True)

Категориальные столбцы для кодирования: ['feat_class', 'feat_group_weeks', 'feat_payment_type', 'feat_payment_place', 'feat_group_format']


In [4]:
corr_df_full_feat = encoded_df_withHWcat.corr()
df_full_feat_cor_vector = corr_df_full_feat.corr()['target'].drop('target')
df_full_feat_cor_vector_we_need = df_full_feat_cor_vector[abs(df_full_feat_cor_vector) > 0.1]

In [6]:
X = encoded_df_withHWcat[df_full_feat_cor_vector_we_need.index]
y = encoded_df_withHWcat['target'].astype('int')

In [7]:
from imblearn.under_sampling import RandomUnderSampler
RUS = RandomUnderSampler(random_state=42)
X, y = RUS.fit_resample(X, y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=42, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.5, random_state=42, stratify=y_test)

In [10]:
kfold = KFold(n_splits=10, shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [12]:
#XGBClassifier
XGB = XGBClassifier(
    device="cuda",
    n_estimators=500,  # Количество деревьев
    learning_rate=0.1,  # Скорость обучения
    max_depth=2,       # Глубина деревье       
    random_state=42
)
XGB.fit(X_train, y_train)
y_pred_XGB = XGB.predict(X_val)
print('Classification_report for XGB\n\n', classification_report(y_val, y_pred_XGB, target_names=['Не расторг','Расторг']))

Classification_report for XGB

               precision    recall  f1-score   support

  Не расторг       0.84      0.85      0.84      1538
     Расторг       0.85      0.84      0.84      1537

    accuracy                           0.84      3075
   macro avg       0.84      0.84      0.84      3075
weighted avg       0.84      0.84      0.84      3075



In [14]:
start = time.time()
print(f'f-1 score: {round(cross_val_score(XGB, X, y, cv=skf, scoring='f1').mean(),3)} \nВремя кроссвалидации: {round(time.time() - start,2)} секунд')

f-1 score: 0.832 
Время кроссвалидации: 2.67 секунд
